In [1]:
import pandas as pd
import yfinance as yf
from sklearn.model_selection import train_test_split

from feature_engineering import *

c:\Python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (5.0.0)/charset_normalizer (2.0.6) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
# Generate X_train, y_train, X_test, y_test for lag={3, 7} and perc={3perc, 5perc}
perc_dict = {'5perc': 0.05, '3perc': 0.03}

for lag in [3, 7]:
    for perc in ['5perc', '3perc']:
        # target column
        target = create_target(perc_dict[perc])
        target = target.replace({'BUY':1, 'HOLD':0, 'SELL':-1})
        target = target.drop(['Adj Close'], axis=1)

        # feature: index price
        prices = yf.download("^GSPC", start="2015-12-01", end="2022-09-02")[['Adj Close']]
        prices = compute_lagged_values(prices, lag, "mean")
        prices = prices.reset_index()
        prices['Date'] = prices['Date'].apply(lambda x: x.date())
        prices = prices.set_index('Date')
        prices.index = pd.DatetimeIndex(prices.index)
        prices = prices[prices.index.isin(target.index)]

        # feature: reddit scores
        reddit_scores = pd.read_excel("data/sentiments/reddit_2016_2022_sentiment_scores.xlsx")
        reddit_scores = reddit_scores.set_index('date')
        reddit_scores = compute_lagged_values(reddit_scores, lag, "mean")
        reddit_scores = reddit_scores[reddit_scores.index.isin(target.index)]
        weight_type = "both" # or "comments", "upvotes"
        reddit_scores = reddit_scores[[f'pos_score_weighted_{weight_type}',f'neg_score_weighted_{weight_type}',f'neu_score_weighted_{weight_type}',f'compound_score_weighted_{weight_type}']]

        # feature: news scores
        nyt_scores = pd.read_excel("data/sentiments/nyt_2016_2022_sentiment_scores.xlsx")
        nyt_scores = nyt_scores.set_index('date')
        nyt_scores = compute_lagged_values(nyt_scores, lag, "mean")
        nyt_scores = nyt_scores[nyt_scores.index.isin(target.index)]

        # feature: macro data
        macro_data = pd.read_excel("data/raw/Macro_Data_2016_to_2022.xlsx")
        macro_data = macro_feature_engineer(macro_data, normalize=False, data_type="both") ### TO DELETE NORMALIZE CODE
        macro_data = macro_data.reindex(target.index)
        macro_data = macro_data[macro_data.index.isin(target.index)]

        # Combine features and target
        data = pd.concat([prices, reddit_scores, nyt_scores, macro_data, target], axis=1)
        
        # Train-test split
        X, y = data.drop(columns={'decision'}), data[['decision']]
        X = X.drop(['Unnamed: 0', 'pos_score', 'neg_score','neu_score','compound_score_weighted_both'], axis = 1)
        X.rename({'pos_score_weighted_both': 'reddit_pos_both', 'neg_score_weighted_both': 'reddit_neg_both', 'neu_score_weighted_both': 'reddit_neu_both', 'pos_weighted':'nyt_pos','neg_weighted':'nyt_neg','neu_weighted':'nyt_neu','Adj Close':'adj_close','Quarterly GDP (Actual)':'quarterly_gdp_actual','Monthly CPI (Actual)':'monthly_cpi_actual','Monthly Short Term Interest Rates (Actual)':'monthly_st_ir_actual','Monthly Unemployment Rate (Actual)':'monthly_unemployment_actual','Quarterly GDP (Growth)':'quarterly_gdp_growth','Monthly CPI (Growth)':'monthly_cpi_growth','Monthly Short Term Interest Rates (Growth)':'monthly_st_ir_growth','Monthly Unemployment Rate (Growth)':'monthly_unemployment_growth'}, axis = 1, inplace = True)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=len(data['2022':]), shuffle=False)

        scaler = StandardScaler() 
        X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
        X_train_scaled = X_train_scaled.set_index(X_train.index)

        X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)
        X_test_scaled = X_test_scaled.set_index(X_test.index)

        X_train_scaled.to_excel(f'data/model_inputs2/x_train_{perc}_lag{lag}.xlsx')
        X_test_scaled.to_excel(f'data/model_inputs2/x_test_{perc}_lag{lag}.xlsx')
        y_train.to_excel(f'data/model_inputs2/y_train_{perc}_lag{lag}.xlsx')
        y_test.to_excel(f'data/model_inputs2/y_test_{perc}_lag{lag}.xlsx')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
pd.read_excel('data/model_inputs2/x_train_5perc_lag7.xlsx')

,Unnamed: 0,adj_close,reddit_pos_both,reddit_neg_both,reddit_neu_both,nyt_pos,nyt_neg,nyt_neu,quarterly_gdp_actual,monthly_cpi_actual,monthly_st_ir_actual,monthly_unemployment_actual,quarterly_gdp_growth,monthly_cpi_growth,monthly_st_ir_growth,monthly_unemployment_growth
0,2016-01-04,-1.242275,1.062823,1.099165,-1.740672,-1.175357,-1.317553,-0.134751,-1.688016,-0.221004,-0.757595,-0.118006,NaN,NaN,NaN,NaN
1,2016-01-05,-1.247554,1.005045,0.889974,-1.532770,-0.494174,-0.032104,0.493425,-1.688016,-0.221004,-0.757595,-0.118006,NaN,NaN,NaN,NaN
2,2016-01-06,-1.257101,0.919084,0.555905,-1.205687,-0.201490,0.148645,1.044987,-1.688016,-0.221004,-0.757595,-0.118006,NaN,NaN,NaN,NaN
3,2016-01-07,-1.271292,0.800313,0.517433,-1.074655,1.108768,0.364538,1.432690,-1.688016,-0.221004,-0.757595,-0.118006,NaN,NaN,NaN,NaN
4,2016-01-08,-1.294047,0.928136,0.530598,-1.193674,1.407321,0.471820,2.278700,-1.688016,-0.221004,-0.757595,-0.118006,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,2021-12-27,2.415950,-0.469680,-1.104095,1.239047,-0.861937,-0.272590,-1.187707,2.474008,0.268338,-1.069440,-0.552311,1.102218,-0.115038,-0.073827,-0.304847
1507,2021-12-28,2.432271,-0.414995,-1.137311,1.216762,-0.968449,-0.793737,-1.247597,2.474008,0.268338,-1.069440,-0.552311,1.102218,-0.115038,-0.073827,-0.304847
1508,2021-12-29,2.455883,-0.355154,-1.165248,1.187546,-1.306063,-0.999733,-1.927244,2.474008,0.268338,-1.069440,-0.552311,1.102218,-0.115038,-0.073827,-0.304847
1509,2021-12-30,2.490478,-0.550838,-1.009204,1.235619,-1.368212,-1.123592,-2.084146,2.474008,0.268338,-1.069440,-0.552311,1.102218,-0.115038,-0.073827,-0.304847


In [2]:
## Target column ##
target_5perc = create_target(0.05)
target_5perc = target_5perc.replace({'BUY':1, 'HOLD':0, 'SELL':-1})

target_3perc = create_target(0.03)
target_3perc = target_3perc.replace({'BUY':1, 'HOLD':0, 'SELL':-1})

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
target = target_3perc.copy(deep=True)
target = target.drop(['Adj Close'], axis=1)

In [4]:
## Feature: Index Price ##
prices = yf.download("^GSPC", start="2015-12-20", end="2022-09-02")[['Adj Close']]
prices = compute_lagged_values(prices, 3, "mean")
prices = prices.reset_index()
prices['Date'] = prices['Date'].apply(lambda x: x.date())

prices = prices.set_index('Date')
prices.index = pd.DatetimeIndex(prices.index)

prices = prices[prices.index.isin(target.index)]

[*********************100%***********************]  1 of 1 completed


In [5]:
## Feature: Reddit Posts ##
# WARNING: This line of code may take up to an hour to run. 
# reddit_posts = pd.read_excel("data/cleaned/reddit_2016_2022_cleaned_1710.xlsx")
# reddit_scores = reddit_feature_engineer(reddit_posts)

# Instead, run this to directly retrieve reddit sentiment scores.
reddit_scores = pd.read_excel("data/sentiments/reddit_2016_2022_sentiment_scores.xlsx")
reddit_scores = reddit_scores.set_index('date')

# Then, compute lagged values and filter rows with target value
reddit_scores = compute_lagged_values(reddit_scores, 3, "mean")
reddit_scores = reddit_scores[reddit_scores.index.isin(target.index)]

weight_type = "both" # or "comments", "upvotes"
reddit_scores = reddit_scores[[f'pos_score_weighted_{weight_type}',f'neg_score_weighted_{weight_type}',f'neu_score_weighted_{weight_type}',f'compound_score_weighted_{weight_type}']]

In [6]:
## Feature: NYT Articles ###
# WARNING: This line of code may take up to an hour to run. 
# nyt_posts = pd.read_excel("data/cleaned/nyt_2016_2022_cleaned_1710.xlsx")
# spweights = pd.read_excel("data/nyt_2016_2022_cleaned_1710")
# nyt_scores = nyt_feature_engineer(nyt_posts, spweights)
# nyt_scores = nyt_scores.set_index('date')

# Instead, run this to directly retrieve nyt sentiment scores.
nyt_scores = pd.read_excel("data/sentiments/nyt_2016_2022_sentiment_scores.xlsx")
nyt_scores = nyt_scores.set_index('date')

# Then, compute lagged values and filter rows with target value
nyt_scores = compute_lagged_values(nyt_scores, 3, "mean")
nyt_scores = nyt_scores[nyt_scores.index.isin(target.index)]

In [7]:
## Feature: Macroeconomic Data ##
macro_data = pd.read_excel("data/raw/Macro_Data_2016_to_2022.xlsx")
macro_data = macro_feature_engineer(macro_data, normalize=False, data_type="both") ### TO DELETE NORMALIZE CODE
macro_data = macro_data.reindex(target.index)
macro_data = macro_data[macro_data.index.isin(target.index)]

In [8]:
# Combine features and target
data = pd.concat([prices, reddit_scores, nyt_scores, macro_data, target], axis=1)

In [9]:
# Train-test split
X, y = data.drop(columns={'decision'}), data[['decision']]
X = X.drop(['Unnamed: 0', 'pos_score', 'neg_score','neu_score','compound_score_weighted_both'], axis = 1)
X.rename({'pos_score_weighted_both': 'reddit_pos_both', 'neg_score_weighted_both': 'reddit_neg_both', 'neu_score_weighted_both': 'reddit_neu_both', 'pos_weighted':'nyt_pos','neg_weighted':'nyt_neg','neu_weighted':'nyt_neu','Adj Close':'adj_close','Quarterly GDP (Actual)':'quarterly_gdp_actual','Monthly CPI (Actual)':'monthly_cpi_actual','Monthly Short Term Interest Rates (Actual)':'monthly_st_ir_actual','Monthly Unemployment Rate (Actual)':'monthly_unemployment_actual','Quarterly GDP (Growth)':'quarterly_gdp_growth','Monthly CPI (Growth)':'monthly_cpi_growth','Monthly Short Term Interest Rates (Growth)':'monthly_st_ir_growth','Monthly Unemployment Rate (Growth)':'monthly_unemployment_growth'}, axis = 1, inplace = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=len(data['2022':]), shuffle=False)

In [10]:
scaler = StandardScaler() 
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
X_train_scaled = X_train_scaled.set_index(X_train.index)

X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)
X_test_scaled = X_test_scaled.set_index(X_test.index)

In [11]:
X_train_scaled.shape, X_test_scaled.shape, y_train.shape, y_test.shape

((1511, 11), (168, 11), (1511, 1), (168, 1))

In [12]:
X_train_scaled.isna().any()

adj_close                      False
reddit_pos_both                False
reddit_neg_both                False
reddit_neu_both                False
nyt_pos                        False
nyt_neg                        False
nyt_neu                        False
quarterly_gdp_actual           False
monthly_cpi_actual             False
monthly_st_ir_actual           False
monthly_unemployment_actual    False
dtype: bool

In [13]:
X_train_scaled

,adj_close,reddit_pos_both,reddit_neg_both,reddit_neu_both,nyt_pos,nyt_neg,nyt_neu,quarterly_gdp_actual,monthly_cpi_actual,monthly_st_ir_actual,monthly_unemployment_actual
2016-01-04,-1.237034,0.750570,0.859067,-1.229261,-0.758713,-0.927853,-0.073841,-1.688016,-0.221004,-0.757595,-0.118006
2016-01-05,-1.267671,0.857737,0.288330,-0.918232,-0.014824,0.378401,0.895025,-1.688016,-0.221004,-0.757595,-0.118006
2016-01-06,-1.289425,0.157307,-0.133683,-0.040935,0.219222,0.827785,0.830477,-1.688016,-0.221004,-0.757595,-0.118006
2016-01-07,-1.314457,0.378284,-0.000596,-0.312078,2.389405,1.557268,2.092301,-1.688016,-0.221004,-0.757595,-0.118006
2016-01-08,-1.346900,0.582352,0.084916,-0.541305,2.768258,0.890287,3.238595,-1.688016,-0.221004,-0.757595,-0.118006
...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,2.440348,-0.357990,-0.617946,0.734099,-1.125003,-1.019085,-1.550085,2.474008,0.268338,-1.069440,-0.552311
2021-12-28,2.506547,0.014023,-0.676437,0.467094,-1.078118,-0.770562,-1.549150,2.474008,0.268338,-1.069440,-0.552311
2021-12-29,2.548418,-0.308494,-0.581064,0.667369,-1.040416,-0.702239,-1.542936,2.474008,0.268338,-1.069440,-0.552311
2021-12-30,2.579788,-0.649372,-0.706349,1.036594,-1.071824,-0.699101,-1.432188,2.474008,0.268338,-1.069440,-0.552311


In [14]:
# X_train_scaled.to_excel('data/model_inputs/x_train_3perc.xlsx')
# X_test_scaled.to_excel('data/model_inputs/x_test_3perc.xlsx')
# y_train.to_excel('data/model_inputs/y_train_3perc.xlsx')
# y_test.to_excel('data/model_inputs/y_test_3perc.xlsx')